### Check the headers of the csv files

In [1]:
! head -n 1 Master.csv | tr ',' '\n'
#! head -n 1 Salaries.csv | tr ',' '\n'

playerID
birthYear
birthMonth
birthDay
birthCountry
birthState
birthCity
deathYear
deathMonth
deathDay
deathCountry
deathState
deathCity
nameFirst
nameLast
nameGiven
weight
height
bats
throws
debut
finalGame
retroID



### Check the content of the csv files

In [2]:
! tail -n +2  Master.csv | head -5
#! tail -n +2  Salaries.csv | head -5






tail: error writing ‘standard output’: Broken pipe


### Extract the data without the headers

In [3]:
! tail -n +2 Master.csv > Master2.csv
! tail -n +2 Salaries.csv > Salaries2.csv

### Put it in HDFS

In [4]:
! hdfs dfs -mkdir -p /user/nasdag/baseball

In [5]:
! hdfs dfs -put Master2.csv /user/nasdag/baseball
! hdfs dfs -put Salaries2.csv /user/nasdag/baseball

### Connect to Hive through JDBC

In [6]:
import pyhs2

In [7]:
conn = pyhs2.connect(host='localhost',
                   port=10000,
                   authMechanism="PLAIN",
                   user='nasdag',
                   password='',
                   database='default')
cur = conn.cursor()

In [8]:
cur.execute("drop table Master")
cur.execute("drop table Salaries")

In [9]:
cur.execute("""CREATE TABLE IF NOT EXISTS Master
      (playerID STRING,
      birthYear INT,
      birthMonth INT,
      birthDay INT,
      birthCountry STRING,
      birthState STRING,
      birthCity STRING,
      deathYear INT,
      deathMonth INT,
      deathDay INT,
      deathCountry STRING,
      deathState STRING,
      deathCity STRING,
      nameFirst STRING,
      nameLast STRING,
      nameGiven STRING,
      weight INT,
      height INT,
      bats STRING,
      throws STRING,
      debut STRING,
      finalGame STRING,
      retroID STRING,
      bbrefID STRING)
      COMMENT 'Master Player Table'
      ROW FORMAT DELIMITED
      FIELDS TERMINATED BY ','
      STORED AS TEXTFILE""")

In [10]:
cur.execute("""CREATE TABLE IF NOT EXISTS Salaries                                                
      (yearID INT, teamID STRING, lgID STRING, playerID STRING, salary INT)              
      COMMENT 'Salary Table for Players'                                                 
      ROW FORMAT DELIMITED                                                               
      FIELDS TERMINATED BY ','                                                           
      STORED AS TEXTFILE""")

In [11]:
cur.execute("select * from default.master")
cur.getSchema()

[{'columnName': 'playerid', 'comment': '', 'type': 'STRING_TYPE'},
 {'columnName': 'birthyear', 'comment': '', 'type': 'INT_TYPE'},
 {'columnName': 'birthmonth', 'comment': '', 'type': 'INT_TYPE'},
 {'columnName': 'birthday', 'comment': '', 'type': 'INT_TYPE'},
 {'columnName': 'birthcountry', 'comment': '', 'type': 'STRING_TYPE'},
 {'columnName': 'birthstate', 'comment': '', 'type': 'STRING_TYPE'},
 {'columnName': 'birthcity', 'comment': '', 'type': 'STRING_TYPE'},
 {'columnName': 'deathyear', 'comment': '', 'type': 'INT_TYPE'},
 {'columnName': 'deathmonth', 'comment': '', 'type': 'INT_TYPE'},
 {'columnName': 'deathday', 'comment': '', 'type': 'INT_TYPE'},
 {'columnName': 'deathcountry', 'comment': '', 'type': 'STRING_TYPE'},
 {'columnName': 'deathstate', 'comment': '', 'type': 'STRING_TYPE'},
 {'columnName': 'deathcity', 'comment': '', 'type': 'STRING_TYPE'},
 {'columnName': 'namefirst', 'comment': '', 'type': 'STRING_TYPE'},
 {'columnName': 'namelast', 'comment': '', 'type': 'STRING_

### Load the data into Hive

In [12]:
cur.execute("LOAD DATA INPATH '/user/nasdag/baseball/Master2.csv' OVERWRITE INTO TABLE Master")
cur.execute("LOAD DATA INPATH '/user/nasdag/baseball/Salaries2.csv' OVERWRITE INTO TABLE Salaries")

In [13]:
cur.execute("SELECT COUNT(playerid) FROM Master")
for i in cur.fetch():
    print i
    break

[18354]


In [14]:
import pandas as pd
cur.execute("SELECT weight, count(playerID) Qty FROM Master GROUP BY weight ORDER BY Qty DESC")
pd.DataFrame(cur.fetchall()).tail()

,0,1
127,127,1
128,128,1
129,132,1
130,133,1
131,134,1


In [15]:
cur = conn.cursor()
cur.execute("""SELECT Salaries.yearID, Master.nameFirst, Master.nameLast, Master.weight, Salaries.salary 
               FROM Master JOIN Salaries ON (Master.playerID = Salaries.playerID) WHERE Master.weight > 270
               ORDER BY salary DESC""")
pd.DataFrame(cur.fetchall()).tail()

,0,1,2,3,4
101,2002,Calvin,Pickering,283,200000
102,2002,Jon,Rauch,290,200000
103,2002,Carlos,Silva,280,200000
104,1997,Dmitri,Young,295,155000
105,1996,Dmitri,Young,295,109000


In [16]:
cur = conn.cursor()
cur.execute("""SELECT * FROM master""")
pd.DataFrame(cur.fetchall()).tail()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
18349,zupcibo01,1966,8,18,USA,PA,Pittsburgh,NaN,NaN,NaN,...,Zupcic,Robert,220,76,R,R,1991-09-07,1994-08-04,zupcb001,zupcibo01
18350,zupofr01,1939,8,29,USA,CA,San Francisco,2005,3,25,...,Zupo,Frank Joseph,182,71,L,R,1957-07-01,1961-05-09,zupof101,zupofr01
18351,zuvelpa01,1958,10,31,USA,CA,San Mateo,NaN,NaN,NaN,...,Zuvella,Paul,173,72,R,R,1982-09-04,1991-05-02,zuvep001,zuvelpa01
18352,zuverge01,1924,8,20,USA,MI,Holland,NaN,NaN,NaN,...,Zuverink,George,195,76,R,R,1951-04-21,1959-06-15,zuveg101,zuverge01
18353,zwilldu01,1888,11,2,USA,MO,St. Louis,1978,3,27,...,Zwilling,Edward Harrison,160,66,L,L,1910-08-14,1916-07-12,zwild101,zwilldu01
